<a href="https://colab.research.google.com/github/mmsamiei/just-practice-deep/blob/master/Abbas_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torchtext
from torchtext.data import Field
import spacy

from spacy.symbols import ORTH
my_tok = spacy.load('en')

def spacy_tok(x):
    return [tok.text for tok in my_tok.tokenizer(x)]

QUERY = Field(lower=True, tokenize=spacy_tok)
RESPONSE = Field(lower=True, tokenize=spacy_tok, is_target=True, init_token='<bos>', eos_token='<eos>')

In [0]:
import torch
if torch.cuda.is_available:
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [0]:
my_tok.tokenizer.add_special_case("don't", [{ORTH: "do"}, {ORTH: "n't"}])

In [0]:
from torchtext.data import TabularDataset

train_dataset = TabularDataset(path="./formatted_movie_lines.txt", format="CSV",
                               fields=[("query", QUERY),("response", RESPONSE)],
                               csv_reader_params={"delimiter":'\t'})

In [5]:
QUERY.build_vocab(train_dataset)
RESPONSE.build_vocab(train_dataset)
print(QUERY.vocab.stoi['film'])
print(QUERY.vocab.itos[33])
print(len(QUERY.vocab))
print(len(RESPONSE.vocab))

1059
're
48505
49036


In [0]:
from torchtext.data import BucketIterator

batch_size = 16

train_iterator = BucketIterator(dataset= train_dataset, batch_size=batch_size
                                ,device=device
                                ,sort_key=lambda x: data.interleave_keys(len(x.query), len(x.response))
                                , repeat = False)

In [7]:
for batch in (iter(train_iterator)):
  print(batch.query.shape)

torch.Size([58, 16])
torch.Size([85, 16])
torch.Size([59, 16])
torch.Size([55, 16])
torch.Size([28, 16])
torch.Size([17, 16])
torch.Size([29, 16])
torch.Size([47, 16])
torch.Size([23, 16])
torch.Size([40, 16])
torch.Size([37, 16])
torch.Size([50, 16])
torch.Size([62, 16])
torch.Size([27, 16])
torch.Size([33, 16])
torch.Size([40, 16])
torch.Size([41, 16])
torch.Size([71, 16])
torch.Size([74, 16])
torch.Size([29, 16])
torch.Size([50, 16])
torch.Size([22, 16])
torch.Size([34, 16])
torch.Size([73, 16])
torch.Size([50, 16])
torch.Size([30, 16])
torch.Size([46, 16])
torch.Size([37, 16])
torch.Size([64, 16])
torch.Size([60, 16])
torch.Size([40, 16])
torch.Size([123, 16])
torch.Size([54, 16])
torch.Size([44, 16])
torch.Size([39, 16])
torch.Size([49, 16])
torch.Size([43, 16])
torch.Size([53, 16])
torch.Size([41, 16])
torch.Size([24, 16])
torch.Size([72, 16])
torch.Size([24, 16])
torch.Size([65, 16])
torch.Size([22, 16])
torch.Size([57, 16])
torch.Size([149, 16])
torch.Size([38, 16])
torch.Size(

In [0]:
import torch.nn as nn
class Bradley(nn.Module):
  def __init__(self, src_voc_sze, trg_voc_sze, hid_sze, num_head, num_enc, num_dec):
    super(Bradley, self).__init__()
    self.hid_sze = hid_sze
    self.src_word_embedding = nn.Embedding(src_voc_sze, self.hid_sze)
    self.trg_word_embedding = nn.Embedding(trg_voc_sze, self.hid_sze)
    self.trg_pos_embedding = nn.Embedding(1000, self.hid_sze)
    self.num_head = num_head
    self.transformer = nn.Transformer(self.hid_sze, self.num_head, num_enc, num_dec)
    self.fc = nn.Linear(self.hid_sze, trg_voc_sze)
  
  def forward(self, src, trg):
    #src = [src sent len, batch_size]
    #trg = [trg sent len, batch_size]
    temp_src = self.src_word_embedding(src)
    temp_trg = self.trg_word_embedding(trg)
    trg_sent_len, batch_size = trg.shape[0], trg.shape[1]
    trg_pos = self.trg_pos_embedding(torch.arange(0, trg_sent_len).unsqueeze(0).
                                     repeat(batch_size,1).to(device)).transpose(0,1)
    trg_mask = self._generate_square_subsequent_mask(trg_sent_len)
    temp_trg = temp_trg + trg_pos
    temp = self.transformer(temp_src, temp_trg, tgt_mask=trg_mask)
    return self.fc(temp)
  
  def _generate_square_subsequent_mask(self, sz):
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    mask = mask.to(device)
    return mask


  def greedy_inference_one_sample(self, src, max_len=10):

    self.eval()
    #src = [src sent len]
    src_len = src.shape[0]
    src = src.unsqueeze(1)
    #src = [sent_len, 1]
    trg = src.new_full((1,1), RESPONSE.vocab.stoi['<bos>'])
    #trg = [1,1]

    translation_step = 0
    while translation_step < max_len:
      out = self.forward(src, trg)
      out = out[-1,:]
      for i, val in enumerate(out[0]):
        if(val > 7):
          print(i, val)
      #out = [batch_size, trg_vocab_size]
      nex = out.argmax(dim=1).unsqueeze(0)
      #nex = [1, 1]
      trg = torch.cat((trg, nex), dim=0)
      translation_step += 1
    return trg



In [0]:
bradley_model = Bradley(src_voc_sze=len(QUERY.vocab), trg_voc_sze=len(RESPONSE.vocab),
                        hid_sze=64, num_head=4,
                        num_enc=3, num_dec=2)
bradley_model = bradley_model.to(device)

In [51]:
from tqdm import tqdm

criterion = nn.CrossEntropyLoss(ignore_index=RESPONSE.vocab.stoi['<pad>'])
lr = 0.5
optimizer = torch.optim.SGD(bradley_model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
epoch_number = 5

print(len(train_dataset)/batch_size)

for epoch in range(1, epoch_number+1):
  print("epoch ", epoch)
  bradley_model.train()
  for i, batch in enumerate(iter(train_iterator)):
    src = batch.query
    trg = batch.response
    optimizer.zero_grad()
    out = bradley_model(src, trg)
    loss = criterion(out[:-1,:].view(-1, out.shape[2]), trg[1:,:].view(-1))
    if(i%50==0):
      print("{}: {}".format(i, loss.item()))
    loss.backward()
    torch.nn.utils.clip_grad_norm_(bradley_model.parameters(), 0.5)
    optimizer.step()

13830.125
epoch  1
0: 4.4117913246154785
50: 4.30262565612793
100: 4.2693772315979
150: 4.7109880447387695
200: 4.4685516357421875
250: 4.599844455718994
300: 4.792407989501953
350: 4.835885047912598
400: 4.381875514984131
450: 4.6439008712768555
500: 4.275861740112305
550: 5.348369598388672
600: 4.329240798950195
650: 5.031351566314697
700: 3.9627859592437744
750: 4.889611721038818
800: 4.7087178230285645
850: 5.255964279174805
900: 5.1995768547058105
950: 5.2042927742004395
1000: 4.4413743019104
1050: 4.46804666519165
1100: 4.593530654907227
1150: 4.692720413208008
1200: 4.887069225311279
1250: 4.93937349319458
1300: 5.0673394203186035
1350: 4.744878768920898
1400: 4.53118371963501
1450: 5.050879955291748
1500: 5.1981329917907715
1550: 5.068643569946289
1600: 4.863820552825928
1650: 4.737473487854004
1700: 5.046177387237549
1750: 4.299705982208252
1800: 4.123277187347412
1850: 4.588231563568115
1900: 4.1836323738098145
1950: 4.527967929840088
2000: 4.485937118530273
2050: 4.588704586

RuntimeError: ignored

In [59]:
source_sentence = train_dataset[50].query
print(source_sentence)

['is', 'he', 'oily', 'or', 'dry', '?']


In [60]:
x = QUERY.numericalize([source_sentence]).to(device)
x = x.flatten()
print(x.shape)
result = bradley_model.greedy_inference_one_sample(x)

torch.Size([6])
6 tensor(11.4878, device='cuda:0', grad_fn=<SelectBackward>)
7 tensor(10.4905, device='cuda:0', grad_fn=<SelectBackward>)
9 tensor(9.8480, device='cuda:0', grad_fn=<SelectBackward>)
12 tensor(8.9482, device='cuda:0', grad_fn=<SelectBackward>)
13 tensor(10.1140, device='cuda:0', grad_fn=<SelectBackward>)
16 tensor(8.7393, device='cuda:0', grad_fn=<SelectBackward>)
17 tensor(9.0812, device='cuda:0', grad_fn=<SelectBackward>)
18 tensor(9.0317, device='cuda:0', grad_fn=<SelectBackward>)
19 tensor(8.9378, device='cuda:0', grad_fn=<SelectBackward>)
20 tensor(7.4783, device='cuda:0', grad_fn=<SelectBackward>)
21 tensor(10.2819, device='cuda:0', grad_fn=<SelectBackward>)
23 tensor(7.9096, device='cuda:0', grad_fn=<SelectBackward>)
24 tensor(7.0524, device='cuda:0', grad_fn=<SelectBackward>)
25 tensor(8.7257, device='cuda:0', grad_fn=<SelectBackward>)
26 tensor(9.2810, device='cuda:0', grad_fn=<SelectBackward>)
27 tensor(9.4862, device='cuda:0', grad_fn=<SelectBackward>)
28 tens

In [64]:
RESPONSE.vocab.itos[120]

'sure'

In [61]:
result

tensor([[  2],
        [  6],
        [ 29],
        [ 36],
        [120],
        [  4],
        [  3],
        [  4],
        [  3],
        [  5],
        [  6]], device='cuda:0')

In [0]:
def batch_index_to_strings(trg):
  # trg = [sent_len, batch_size]
  temp = trg.transpose(0,1)
  for i, row in enumerate(temp):
    print(row)

batch_index_to_strings(result)

In [0]:
mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

In [0]:
mask = (torch.triu(torch.ones(5, 5)) == 1).transpose(0, 1)
mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

In [0]:
mask